In [ ]:
!pip3 install -r requirements.txt

In [2]:
from dotenv import dotenv_values
import os
SECRETS=dotenv_values(".env")
os.environ['OPENAI_API_KEY'] = SECRETS['OPENAI_API_KEY']

# Load Data & Embeddings

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

file_path = '/Users/rayanaay/Desktop/projects/langchain/summarizing_project/mix_data/rust_essentials.pdf'

loader = PyPDFLoader(file_path=file_path)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0
)

data = loader.load_and_split(text_splitter=text_splitter)
embeddings = OpenAIEmbeddings(
    openai_api_key=SECRETS["OPENAI_API_KEY"] ,
    show_progress_bar=True)


# Loading data into the vectorDB

## Create a Pinecone Index

In [4]:
from pinecone import Pinecone as PineconeClient, ServerlessSpec
import os

pc = PineconeClient(api_key=SECRETS["PINECONE_API_KEY"])

index_name = "my-vector-db"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536, 
        metric="cosine", 
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        ) 
    ) 
else:
    print("already exists")

/Users/rayanaay/anaconda3/envs/langchain/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


already exists


## Feed the index with data

Pinecone allows you to partition the records in an index into namespaces. Queries and other operations are then limited to one namespace, so different requests can search different subsets of your index.

In [5]:
from langchain_pinecone import PineconeVectorStore

namespace = "my-namespace"

docsearch = PineconeVectorStore.from_documents(
    documents=data,
    index_name=index_name,
    embedding=embeddings, 
    namespace=namespace 
)

100%|██████████| 1/1 [00:03<00:00,  3.60s/it]


# Use your Pinecone index for RAG

Now that your data is stored in your vector database, we can retrieve it

In [8]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA  

llm = ChatOpenAI(temperature=0.0)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=docsearch.as_retriever()
)

In [9]:
chain.run(
    'What is concurrency in Rust?',
)

/Users/rayanaay/anaconda3/envs/langchain/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
100%|██████████| 1/1 [00:00<00:00,  3.89it/s]


"Concurrency in Rust refers to the ability of the language to execute multiple tasks or processes simultaneously. Rust provides a variety of tools and features for managing concurrency, allowing developers to write code that can run efficiently on multiple processors at the same time. Rust's strong type system enables the creation of concurrency primitives with unique properties, making it a powerful language for handling concurrent execution."

## Delete Pinecone Index

In [10]:
pc.delete_index(index_name)